Lukas notebook

In [1]:
!pwd

/Users/lukasfiller/dev/browser-use


In [ ]:
%cd dev
%cd browser-use

/Users/lukasfiller/dev
/Users/lukasfiller/dev/browser-use


In [ ]:
#optional

!git clone https://github.com/browser-use/browser-use
%cd browser-use

/Users/lukasfiller/dev/browser-use


In [ ]:
#creates env
!uv venv --python 3.11

zsh:1: command not found: uv


In [ ]:
#reactivates the env
!source .venv/bin/activate

In [2]:
#run in cli activated .venv as
#pip install -U browser-use

#or here...run this in the conda env
!uv pip install -U browser-use

Using Python 3.12.9 environment at: /Users/lukasfiller/miniforge3/envs/crawl4ai
Resolved 51 packages in 2.38s                                        
⠙ Preparing packages... (0/33)                                                  
⠙ Preparing packages... (0/33)---     0 B/125.66 KiB                    
posthog    ------------------------------     0 B/87.03 KiB
⠙ Preparing packages... (0/33)---     0 B/125.66 KiB                    
httpcore   ------------------------------     0 B/76.89 KiB
posthog    ------------------------------     0 B/87.03 KiB
⠙ Preparing packages... (0/33)---     0 B/125.66 KiB                    
requests-toolbelt ------------------------------     0 B/53.20 KiB
httpcore   ------------------------------     0 B/76.89 KiB
posthog    ------------------------------     0 B/87.03 KiB
⠙ Preparing packages... (0/33)---     0 B/125.66 KiB                    
tenacity   ------------------------------     0 B/27.59 KiB
requests-toolbelt ------------------------------  

In [3]:
!pip install -U langchain-ollama

  Attempting uninstall: langchain-ollama
    Found existing installation: langchain-ollama 0.2.2
    Uninstalling langchain-ollama-0.2.2:
      Successfully uninstalled langchain-ollama-0.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
browser-use 0.1.40 requires langchain-ollama==0.2.2, but you have langchain-ollama 0.3.2 which is incompatible.


In [4]:
!pip install -U playwright

In [5]:
!playwright install

Removing unused browser at /Users/lukasfiller/Library/Caches/ms-playwright/chromium_headless_shell-1155
124.1 MiB [                    ] 0% 0.0s124.1 MiB [                    ] 0% 163.1s124.1 MiB [                    ] 0% 446.8s124.1 MiB [                    ] 0% 330.0s124.1 MiB [                    ] 0% 299.5s124.1 MiB [                    ] 0% 308.9s124.1 MiB [                    ] 0% 253.2s124.1 MiB [                    ] 0% 212.1s124.1 MiB [                    ] 0% 183.2s124.1 MiB [                    ] 0% 161.2s124.1 MiB [                    ] 0% 146.1s124.1 MiB [                    ] 0% 128.4s124.1 MiB [                    ] 0% 114.7s124.1 MiB [                    ] 0% 110.2s124.1 MiB [                    ] 0% 87.0s124.1 MiB [                    ] 0% 77.6s124.1 MiB [                    ] 0% 74.0s124.1 MiB [                    ] 0% 65.1s124.1 MiB [                    ] 0% 54.5s124.1 MiB [                    ] 0% 46.0s124.1 MiB [                    ] 0% 42.0s124.1 MiB [            

In [6]:

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

In [ ]:
#not needed when running locally?
!pip install python-dotenv

from dotenv import load_dotenv
load_dotenv()

In [ ]:
# -*- coding: utf-8 -*-
import os
import sys
import asyncio
from dotenv import load_dotenv
from langchain_ollama import ChatOllama # Using Ollama

# --- Ensure browser_use is installed or correctly on sys.path ---
try:
    from browser_use import Agent, Browser, BrowserConfig # Import Browser and Config
    print("Successfully imported browser_use components.")
except ImportError as e:
    print(f"Error importing browser_use: {e}")
    print("Please ensure 'browser_use' is installed in your Jupyter kernel's environment.")
    print("You might need to run: pip install browser-use  (or similar, depending on the package name)")
    # Or ensure the path is correct if it's a local module
    # cwd = os.getcwd()
    # print(f"Current working directory: {cwd}")
    # print(f"Python path: {sys.path}")
    raise # Stop execution if import fails

# Load environment variables if needed
load_dotenv()
print("Environment variables loaded (if .env file exists).")

# Initialize the model using Ollama
# --- Make sure your Ollama service is running locally ---
llm = None # Initialize llm to None
try:
    llm = ChatOllama(
        model='phi4',        # Or 'llama3', 'mistral', etc.
        temperature=0.0,
    )
    print(f"Successfully initialized Ollama model: {llm.model}")
    # Optional: Quick test
    # llm.invoke("test connection")
except Exception as e:
    print(f"Error initializing Ollama model: {e}")
    print("Please ensure the Ollama service is running and the model is available.")
    raise # Stop execution if LLM fails

# --- Explicit Browser Configuration ---
# Note: Playwright browsers might need to be installed ('playwright install' in terminal/cell)
browser = None # Initialize browser to None
try:
    print("Configuring browser...")
    config = BrowserConfig(
        headless=True,  # Run without a visible browser window
        # user_agent="...", # Optional
        # viewport={"width": 1920, "height": 1080}, # Optional
    )
    print("Initializing browser...")
    # This step might download browser drivers on first run
    browser = Browser(config=config)
    print("Browser initialized.")
except Exception as e:
    print(f"Error initializing Browser: {e}")
    print("This might be due to Playwright browsers not being installed.")
    print("Try running '!playwright install' in a cell or 'playwright install' in your terminal.")
    raise # Stop execution if browser fails

# Define the task for the agent
current_year = 2025 # Based on user context
future_date = f"{current_year + 1}-05-01" # Use next year for the search
task = f'Go to kayak.com and find the cheapest flight from Zurich (ZRH) to San Francisco (SFO) departing on {future_date}'
print(f"Task defined: {task}")

# Initialize the Agent - *Pass the browser instance*
print("Initializing Agent...")
if llm and browser:
    agent = Agent(task=task, llm=llm, browser=browser) # Pass browser here
    print("Agent initialized.")
else:
    print("Cannot initialize Agent due to previous errors.")
    raise ValueError("LLM or Browser not initialized successfully.")


# Define the main asynchronous function
async def main():
    print("Running agent asynchronously...")
    result = None # Initialize result
    try:
        result = await agent.run()
        print("Agent run completed.")
        # Process result
        if result and hasattr(result, 'action_results'):
            print("Processing actions...")
            final_content = []
            for action in result.action_results():
                print(f"  Action: {action.action_type}, Status: {action.status}")
                if action.extracted_content:
                    content_snippet = action.extracted_content[:200].replace('\n', ' ') + ('...' if len(action.extracted_content) > 200 else '')
                    print(f"    Extracted: {content_snippet}")
                    final_content.append(action.extracted_content)
            print("\n--- Combined Extracted Content ---")
            print("\n\n".join(final_content) if final_content else "No content extracted.")
            print("--------------------------------")

        elif result:
            print(f"Agent Result (raw): {result}")
        else:
            print("Agent did not return a result.")

    except Exception as e:
        print(f"An error occurred during agent execution: {e}")
        # You might want to re-raise the exception for debugging
        import traceback
        traceback.print_exc()
        # raise
    finally:
        # Ensure the browser used by the agent is closed
        # Since we passed the browser instance, we can close it directly
        if browser and hasattr(browser, 'close'):
            print("Attempting to close browser...")
            try:
                await browser.close()
                print("Browser closed.")
            except Exception as close_e:
                print(f"Error closing browser: {close_e}")
        else:
            print("Browser instance not available for cleanup.")

# Run the main async function in Jupyter
# No need for 'if __name__ == "__main__":' when running directly in a cell
print("Starting async execution...")
try:
    # In modern Jupyter/IPython, you might be able to just 'await main()'
    # but asyncio.run() is generally safer for compatibility.
    asyncio.run(main())
    print("Async execution finished successfully.")
except Exception as e:
    print(f"An critical error occurred running main: {e}")
    # Print traceback if error happens here
    import traceback
    traceback.print_exc()
finally:
    print("Execution block finished.")

Successfully imported browser_use components.
Environment variables loaded (if .env file exists).
Successfully initialized Ollama model: phi4
Configuring browser...
Initializing browser...
DEBUG    [browser] 🌎  Initializing new browser
Browser initialized.
Task defined: Go to kayak.com and find the cheapest flight from Zurich (ZRH) to San Francisco (SFO) departing on 2026-05-01
Initializing Agent...
DEBUG    [telemetry] Telemetry event: controller_registered_functions {'registered_functions': [{'name': 'done', 'params': {'properties': {'text': {'title': 'Text', 'type': 'string'}, 'success': {'title': 'Success', 'type': 'boolean'}}, 'required': ['text', 'success'], 'title': 'DoneAction', 'type': 'object'}}, {'name': 'search_google', 'params': {'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'SearchGoogleAction', 'type': 'object'}}, {'name': 'go_to_url', 'params': {'properties': {'url': {'title': 'Url', 'type': 'string'}}, 'required': ['url'

Traceback (most recent call last):
  File "/var/folders/28/n_1h_b8x7c9fnqytqt4ldygr0000gn/T/ipykernel_32948/1538463029.py", line 129, in <module>
    asyncio.run(main())
  File "/Users/lukasfiller/miniforge3/envs/crawl4ai/lib/python3.12/asyncio/runners.py", line 191, in run
    raise RuntimeError(
RuntimeError: asyncio.run() cannot be called from a running event loop
/var/folders/28/n_1h_b8x7c9fnqytqt4ldygr0000gn/T/ipykernel_32948/1538463029.py:135: RuntimeWarning: coroutine 'main' was never awaited
  traceback.print_exc()


In [ ]:
#ollama building block only
from langchain_ollama import ChatOllama
from browser_use import Agent
from pydantic import SecretStr


# Initialize the model
llm=ChatOllama(model="phi4", num_ctx=32000)
planner_llm = ChatOllama(model='phi4')

# Create agent with the model
# Create agent with the model
agent = Agent(
    task='''research how the digital china program is seeking to advance china
    AI capabilities. Use only info found at https://digitalchinawinsthefuture.com/''',
    llm=llm,
    planner_llm=planner_llm,           # Separate model for planning
    use_vision_for_planner=False,      # Disable vision for planner
    planner_interval=4
)

In [2]:
#try 16apr2025

# -*- coding: utf-8 -*-
# Imports
import asyncio
from langchain_ollama import ChatOllama # Use Ollama
# from pydantic import SecretStr # Import if needed for other configurations, unused here
from browser_use import Agent, Browser, BrowserConfig

# Initialize LLMs using local Ollama phi4
# Ensure Ollama service is running locally
try:
    print("Initializing LLMs with Ollama (phi4)...")
    llm = ChatOllama(model="phi4", num_ctx=32000)
    planner_llm = ChatOllama(model='phi4')
    print("LLMs initialized successfully.")
    # Optional: You might want to run a quick test invocation here if needed
    # llm.invoke("test")
    # planner_llm.invoke("test")
except Exception as e:
    print(f"Error initializing Ollama models: {e}")
    print("Please ensure the Ollama service is running and the 'phi4' model is available.")
    # Exit or raise error if LLMs are critical and failed to load
    raise

# Basic configuration for the browser
print("Configuring browser...")
config = BrowserConfig(
    headless=False,  # Run in headless mode (no visible browser window)
    # disable_security=True  # Uncomment if you encounter security policy issues
    # user_agent="Mozilla/5.0 ...", # Optional: Set a specific user agent if needed
    # viewport={"width": 1920, "height": 1080}, # Optional: Set viewport size
)

# Initialize the browser with the specified configuration
# Note: This might download a browser executable on first run
print("Initializing browser...")
browser = Browser(config=config)
print("Browser initialized.")

# Define the main asynchronous function
async def main():
    print("Setting up Agent...")
    # Initialize the agent with the specific task, LLMs, browser, and new parameters
    agent = Agent(
        task='''research how the digital china program is seeking to advance china AI capabilities. Use only info found at https://digitalchinawinsthefuture.com/''',
        llm=llm,                     # Main LLM (Ollama phi4)
        planner_llm=planner_llm,     # Planner LLM (Ollama phi4)
        browser=browser,             # The initialized browser instance
        use_vision_for_planner=False,# Disable vision for the planner LLM
        planner_interval=4,          # Set planner interval
        generate_gif=False,          # Disable GIF generation (from original template)
        # max_actions=10,            # Optional: Limit the number of actions agent can take
        # verbose=True,              # Optional: Enable more detailed logging from the agent
    )
    print("Agent setup complete.")

    # Run the agent and get results asynchronously
    print(f"Running agent for task: '{agent.task[:100]}...'") # Print start of task
    try:
        result = await agent.run()
        print("Agent run finished. Processing results...")
    except Exception as e:
        print(f"An error occurred during agent execution: {e}")
        # Ensure browser is closed even if agent fails
        print("Attempting to close browser after error...")
        await browser.close()
        print("Browser closed.")
        return # Exit main function early

    # Process results
    final_output = []
    if result and hasattr(result, 'action_results'):
        print("\n--- Agent Actions and Extracted Content ---")
        for i, action in enumerate(result.action_results()):
            print(f"\nAction {i+1}:")
            print(f"  Type: {action.action_type}")
            print(f"  Status: {action.status}")
            if hasattr(action, 'page_content') and action.page_content:
                 # Limit printing potentially large page content
                 print(f"  Page Content Snippet: {action.page_content[:200]}...")
            if action.extracted_content:
                print(f"  Extracted Content:\n---\n{action.extracted_content}\n---")
                final_output.append(action.extracted_content)
            # Optional: Check if the agent signals it's done
            # if action.is_done:
            #     print("Agent signaled completion.")
            #     break
        print("\n--- End of Actions ---")
    else:
        print("No results or action_results found.")

    # Print combined final output
    print("\n--- Combined Extracted Content ---")
    if final_output:
        print("\n\n".join(final_output))
    else:
        print("No content was extracted.")
    print("--------------------------------")

    # Close the browser after completion
    print("Closing browser...")
    await browser.close()
    print("Browser closed.")


# Run the asynchronous main function using asyncio
# Using if __name__ == "__main__": is good practice for notebooks/scripts
if __name__ == "__main__":
    print("Starting asynchronous execution...")
    try:
        asyncio.run(main())
        print("Execution finished successfully.")
    except Exception as e:
        print(f"An critical error occurred during execution: {e}")
        # Attempt graceful shutdown if possible, though browser closing is handled in main()
    finally:
        # This block executes whether an exception occurred or not
        # Might be useful for other cleanup if needed
        print("Script finished.")

Initializing LLMs with Ollama (phi4)...
LLMs initialized successfully.
Configuring browser...
Initializing browser...
DEBUG    [browser] 🌎  Initializing new browser
Browser initialized.
Starting asynchronous execution...
Setting up Agent...
DEBUG    [telemetry] Telemetry event: controller_registered_functions {'registered_functions': [{'name': 'done', 'params': {'properties': {'text': {'title': 'Text', 'type': 'string'}, 'success': {'title': 'Success', 'type': 'boolean'}}, 'required': ['text', 'success'], 'title': 'DoneAction', 'type': 'object'}}, {'name': 'search_google', 'params': {'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'SearchGoogleAction', 'type': 'object'}}, {'name': 'go_to_url', 'params': {'properties': {'url': {'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'title': 'GoToUrlAction', 'type': 'object'}}, {'name': 'go_back', 'params': {'additionalProperties': True, 'description': 'Accepts absolutely anything in the i

/Users/lukasfiller/miniforge3/envs/crawl4ai/lib/python3.12/site-packages/ollama/_types.py:81: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if key in self.model_fields:
/Users/lukasfiller/miniforge3/envs/crawl4ai/lib/python3.12/site-packages/ollama/_types.py:82: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields[key].default is not None


INFO     [agent] 🤷 Eval: Unknown - The browser was started, but no significant actions have been taken yet to progress towards the ultimate task.
INFO     [agent] 🧠 Memory: Initial setup complete. Browsing session began with an empty tab ('about:blank'). No steps towards the task of researching China's AI capabilities under the digital china program have been performed yet.
INFO     [agent] 🎯 Next goal: Navigate to https://digitalchinawinsthefuture.com/
INFO     [agent] 🛠️  Action 1/1: {"go_to_url":{"url":"https://digitalchinawinsthefuture.com/"}}
DEBUG    [browser_use] --get_next_action (agent) Execution time: 8.72 seconds
DEBUG    [browser_use] --remove_highlights Execution time: 0.01 seconds
DEBUG    [browser_use] --act Execution time: 0.00 seconds
INFO     [controller] 🔗  Navigated to https://digitalchinawinsthefuture.com/
DEBUG    [browser_use] --execute_action Execution time: 3.60 seconds
DEBUG    [agent] Executed action 1 / 1
DEBUG    [browser_use] --multi-act (agent) Execution 

/Users/lukasfiller/miniforge3/envs/crawl4ai/lib/python3.12/site-packages/ollama/_types.py:81: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if key in self.model_fields:
/Users/lukasfiller/miniforge3/envs/crawl4ai/lib/python3.12/site-packages/ollama/_types.py:82: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields[key].default is not None


INFO     [agent] 👍 Eval: Success. The previous goal of navigating to https://digitalchinawinsthefuture.com/ was successfully achieved, as confirmed by the current URL.
INFO     [agent] 🧠 Memory: Navigation to https://digitalchinawinsthefuture.com/ completed in step 1 out of 100. The page contains multiple links and content related to 'Digital China'. No specific information extracted yet about how the Digital China program advances AI capabilities.
INFO     [agent] 🎯 Next goal: Examine the content on the page for relevant information regarding China's AI advancements through the Digital China program.
INFO     [agent] 🛠️  Action 1/1: {"extract_content":{"goal":"Identify sections related to AI and digital strategy.","should_strip_link_urls":true}}
DEBUG    [browser_use] --get_next_action (agent) Execution time: 9.72 seconds
DEBUG    [browser_use] --remove_highlights Execution time: 0.01 seconds
DEBUG    [browser_use] --act Execution time: 0.00 seconds


...

INFO     [agent] 👍 Eval: Success. The current page was successfully navigated, and relevant data on China's AI capabilities were extracted from the content.
INFO     [agent] 🧠 Memory: Information about Digital China, including its strategic goals and technological advancements such as big data, cloud computing, and artificial intelligence, has been collected. Approximately 4% progress towards understanding the broader implications of China's digital strategy has been made by identifying key articles and quotes relevant to AI capabilities.
INFO     [agent] 🎯 Next goal: Continue exploring the page for additional sections or articles that specifically address the advancement of China's AI capabilities within the Digital China initiative.
INFO     [agent] 🛠️  Action 1/1: {"extract_content":{"goal":"Look for further details on China's strategies and initiatives in advancing AI capabilities, such as specific government programs, technological innovations, research efforts, international coll

/Users/lukasfiller/miniforge3/envs/crawl4ai/lib/python3.12/site-packages/ollama/_types.py:81: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if key in self.model_fields:
/Users/lukasfiller/miniforge3/envs/crawl4ai/lib/python3.12/site-packages/ollama/_types.py:82: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields[key].default is not None


INFO     [controller] 📄  Extracted from page
: ```json
{
  "China_AI_Strategies_Initiatives": {
    "Overview": "Over two decades, Xi Jinping developed the 'Digital China' strategy, focusing on comprehensive digital transformation at a national level. This initiative aims to enhance China's core competitiveness and societal efficiency through advancements in technology, including AI.",
    "KeyComponents": [
      {
        "NationalDataAdministration": {
          "Description": "Established in October 2023, this department is tasked with advancing the planning and building of Digital China, which includes creating a digital economy, society, government, ecology, and modern industrial system."
        }
      },
      {
        "DigitalChinaDevelopmentReport": {
          "Year": "2021",
          "Focus": "Emphasizes new generation information technologies like big data, cloud computing, and artificial intelligence. AI is recognized as a crucial strategic resource for the digital age

/Users/lukasfiller/miniforge3/envs/crawl4ai/lib/python3.12/site-packages/ollama/_types.py:81: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if key in self.model_fields:
/Users/lukasfiller/miniforge3/envs/crawl4ai/lib/python3.12/site-packages/ollama/_types.py:82: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields[key].default is not None


INFO     [agent] 👍 Eval: Success. The relevant information about China's AI capabilities has been extracted from the 'Your Guide to Digital China: Where to Begin' page.
INFO     [agent] 🧠 Memory: The task of extracting information related to advancing China's AI capabilities was completed by analyzing the content on the page titled 'Your Guide to Digital China: Where to Begin.' Key insights were synthesized regarding technological advancements, strategic initiatives, and international influence. This step is complete, with 1 out of 1 steps done.
INFO     [agent] 🎯 Next goal: Complete the ultimate task by synthesizing all gathered information and preparing a final summary using the 'done' action.
INFO     [agent] 🛠️  Action 1/1: {"done":{"text":"The Digital China initiative under Xi Jinping focuses on comprehensive digital transformation at a national level. This includes key technological advancements in AI, with strategies emphasizing the development of new generation information tech

In [ ]:
from langchain_ollama import ChatOllama
from browser_use import Agent
from pydantic import SecretStr


# Initialize the model
llm=ChatOllama(model="qwen2.5:14b", num_ctx=32000)
planner_llm = ChatOllama(model='qwen2.5:14b')

async def run_search() -> AgentHistoryList:
    agent = Agent(
        task='''research how the digital china program is seeking to advance china
        AI capabilities. Use only info found at https://digitalchinawinsthefuture.com/''',
        llm=llm,
        planner_llm=planner_llm,           # Separate model for planning
        use_vision_for_planner=False,      # Disable vision for planner
        planner_interval=4
    )

    result = await agent.run()
    return result


async def main():
    result = await run_search()
    print("\n\n", result)


if __name__ == "__main__":
    # Instead of asyncio.run(main()), use the following:
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main()) # Now this should work within your existing event loop

In [ ]:
from langchain_openai import ChatOpenAI
from browser_use import Agent
from dotenv import load_dotenv
load_dotenv()

import asyncio

llm = ChatOpenAI(model="gpt-4o")

async def main():
    agent = Agent(
        task="Compare the price of gpt-4o and DeepSeek-V3",
        llm=llm,
    )
    result = await agent.run()
    print(result)

asyncio.run(main())

In [ ]:
# online example

# -*- coding: utf-8 -*-
import os
import sys
import asyncio
from dotenv import load_dotenv
from langchain_ollama import ChatOllama # Using Ollama

# --- Ensure browser_use is installed or correctly on sys.path ---
try:
    from browser_use import Agent, Browser, BrowserConfig # Import Browser and Config
    print("Successfully imported browser_use components.")
except ImportError as e:
    print(f"Error importing browser_use: {e}")
    print("Please ensure 'browser_use' is installed in your Jupyter kernel's environment.")
    print("You might need to run: pip install browser-use  (or similar, depending on the package name)")
    raise # Stop execution if import fails

# Load environment variables if needed
load_dotenv()
print("Environment variables loaded (if .env file exists).")

# Initialize the model using Ollama
# --- Make sure your Ollama service is running locally ---
llm = None # Initialize llm to None
try:
    llm = ChatOllama(
        model='phi4',        # Or 'llama3', 'mistral', etc.
        temperature=0.0,
    )
    print(f"Successfully initialized Ollama model: {llm.model}")
except Exception as e:
    print(f"Error initializing Ollama model: {e}")
    print("Please ensure the Ollama service is running and the model is available.")
    raise # Stop execution if LLM fails

# --- Explicit Browser Configuration ---
browser = None # Initialize browser to None
try:
    print("Configuring browser...")
    config = BrowserConfig(
        headless=False,  # Run without a visible browser window
    )
    print("Initializing browser...")
    browser = Browser(config=config)
    print("Browser initialized.")
except Exception as e:
    print(f"Error initializing Browser: {e}")
    print("This might be due to Playwright browsers not being installed.")
    print("Try running '!playwright install' in a cell or 'playwright install' in your terminal.")
    raise # Stop execution if browser fails

# Define the task for the agent
current_year = 2025 # Based on user context
future_date = f"{current_year + 1}-05-01" # Use next year for the search
task = f'Go to kayak.com and find the cheapest flight from Zurich (ZRH) to San Francisco (SFO) departing on {future_date}'
print(f"Task defined: {task}")

# Initialize the Agent - *Pass the browser instance*
print("Initializing Agent...")
if llm and browser:
    agent = Agent(task=task, llm=llm, browser=browser) # Pass browser here
    print("Agent initialized.")
else:
    print("Cannot initialize Agent due to previous errors.")
    raise ValueError("LLM or Browser not initialized successfully.")


# Define the main asynchronous function
async def main():
    print("Running agent asynchronously...")
    result = None # Initialize result
    try:
        result = await agent.run()
        print("Agent run completed.")
        # Process result
        if result and hasattr(result, 'action_results'):
            print("Processing actions...")
            final_content = []
            for action in result.action_results():
                print(f"  Action: {action.action_type}, Status: {action.status}")
                if action.extracted_content:
                    content_snippet = action.extracted_content[:200].replace('\n', ' ') + ('...' if len(action.extracted_content) > 200 else '')
                    print(f"    Extracted: {content_snippet}")
                    final_content.append(action.extracted_content)
            print("\n--- Combined Extracted Content ---")
            print("\n\n".join(final_content) if final_content else "No content extracted.")
            print("--------------------------------")

        elif result:
            print(f"Agent Result (raw): {result}")
        else:
            print("Agent did not return a result.")

    except Exception as e:
        print(f"\n--- An error occurred during agent execution ---")
        print(f"Error type: {type(e).__name__}")
        print(f"Error details: {e}")
        print("Traceback:")
        import traceback
        traceback.print_exc()
        print("-------------------------------------------")
        # Decide if you want to re-raise the error after logging
        # raise
    finally:
        # Ensure the browser used by the agent is closed
        if browser and hasattr(browser, 'close'):
            print("Attempting to close browser...")
            try:
                await browser.close()
                print("Browser closed.")
            except Exception as close_e:
                print(f"Error closing browser: {close_e}")
        else:
            # This case shouldn't happen if browser was initialized successfully before main()
            print("Browser instance not available for cleanup in main's finally block.")

# --- Run the main async function directly in Jupyter's event loop ---

print("Starting async execution (using await main())...")
try:
    # Directly await the main coroutine here, as Jupyter has a running event loop
    await main()
    print("Async execution finished successfully.")

except Exception as e:
    # This catches errors happening *outside* the main() function's try/except
    # (e.g., during initialization before main is called, though we have try/except blocks there too)
    # Or if main() re-raises an exception.
    print(f"\n--- An critical error occurred outside or re-raised from main ---")
    print(f"Error type: {type(e).__name__}")
    print(f"Error details: {e}")
    print("Traceback:")
    import traceback
    traceback.print_exc()
    print("-------------------------------------------------------------")

finally:
    # Final message after all execution attempt
    print("Execution block finished.")
#import os

# Optional: Disable telemetry
# os.environ["ANONYMIZED_TELEMETRY"] = "false"

# Optional: Set the OLLAMA host to a remote server
#os.environ["OLLAMA_HOST"] = "http://127.0.0.1:11434"

import asyncio
from browser_use import Agent
from browser_use.agent.views import AgentHistoryList
from langchain_ollama import ChatOllama


async def run_search() -> AgentHistoryList:
    agent = Agent(
        task="Search for a 'browser use' post on the r/LocalLLaMA subreddit and open it.",
        llm=ChatOllama(
            model="qwen2.5:14b",
            num_ctx=32000,
        ),
    )

    result = await agent.run()
    return result


async def main():
    result = await run_search()
    print("\n\n", result)


if __name__ == "__main__":
    # Instead of asyncio.run(main()), use the following:
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main()) # Now this should work within your existing event loop

In [ ]:
!ollama run qwen2.5:14b "What is the capital of France?"

⠙ ⠹ ⠸ ⠼ ⠴ ⠴ ⠧ ⠇ ⠏ ⠋ ⠋ ⠙ ⠹ ⠸ ⠼ ⠦ ⠧ ⠧ ⠏ ⠏ ⠙ ⠹ ⠹ ⠼ ⠼ ⠦ ⠦ ⠇ ⠇ ⠋ ⠋ ⠹ ⠸ ⠼ ⠴ ⠦ ⠦ ⠧ The capital of France is Paris.



In [ ]:
agent = Agent(
    task="your task",
    llm=llm,
    controller=custom_controller,  # For custom tool calling
    use_vision=True,              # Enable vision capabilities
    save_conversation_path="logs/conversation.json"  # Save chat logs
)

In [ ]:
from langchain_openai import ChatOpenAI

# Initialize models
llm = ChatOpenAI(model='gpt-4o')
planner_llm = ChatOpenAI(model='o3-mini')

agent = Agent(
    task="your task",
    llm=llm,
    planner_llm=planner_llm,           # Separate model for planning
    use_vision_for_planner=False,      # Disable vision for planner
    planner_interval=4                 # Plan every 4 steps
)

In [ ]:
!pip install -U langchain_groq

In [ ]:
#use groq
import os
from dotenv import load_dotenv
import asyncio
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

from browser_use import Agent
from browser_use.browser.browser import Browser
from browser_use.browser.browser import BrowserConfig
from browser_use.browser.context import BrowserContextConfig, BrowserContextWindowSize

load_dotenv()

os.environ["GROQ_API_KEY"] = "XXXXX"
# Initialize Groq client
#llm=ChatGroq(model='llama-3.3-70b-versatile', api_key=api_key)

llm=ChatGroq(model='llama3-70b-8192', api_key=api_key)

async def run_agent():
    config = BrowserConfig(headless=True,
                           disable_security=True)
    browser = Browser(config)

    agent = Agent(
        task="Who is Lukas Filler?",
        llm=llm,
        browser=browser,
        generate_gif=False,
        use_vision=False
)
    result = await agent.run()
    print("Agent result:", result)

# Run agent asynchronously
asyncio.run(run_agent())


In [ ]:
print("Agent result:", result)


NameError: name 'result' is not defined